In [1]:
!pip install numpy
!pip install dlib
!pip install pygame
!pip install opencv-python==4.6.0.66 opencv-contrib-python==4.6.0.66






In [2]:
import cv2
print(cv2.__version__)


4.6.0


In [2]:
import numpy as np
import dlib
import cv2
import time

In [ ]:
RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))

frame_width = 640
frame_height = 480

title_name = 'Drowsiness Detection'

face_cascade_name = '/Users/gimseongmin/Downloads/opencv-4.x/data/haarcascades/haarcascade_frontalface_alt.xml' #-- 본인 환경에 맞게 변경할 것
face_cascade = cv2.CascadeClassifier()
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('--(!)Error loading face cascade')
    exit(0)

predictor_file = '/Users/gimseongmin/Downloads/shape_predictor_68_face_landmarks (1).dat' #-- 본인 환경에 맞게 변경할 것
predictor = dlib.shape_predictor(predictor_file)

status = 'Awake'
number_closed = 0
min_EAR = 0.25
closed_limit = 35  # 눈 감김이 35번 이상일 경우 변수를 넘기고 다시 카운트 시작
drowsiness_count = 0  # 졸음 카운트 변수
show_frame = None
sign = None
color = (0, 255, 0)  # 기본 색상 설정

def getEAR(points):
    A = np.linalg.norm(points[1] - points[5])
    B = np.linalg.norm(points[2] - points[4])
    C = np.linalg.norm(points[0] - points[3])
    return (A + B) / (2.0 * C)

def detectAndDisplay(image):
    global number_closed
    global drowsiness_count
    global color
    global show_frame
    global sign
    global status
    global RIGHT_EYE
    global LEFT_EYE
    global EYES

    image = cv2.resize(image, (frame_width, frame_height))
    show_frame = image
    frame_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # CLAHE 적용(컴퓨터 비전 전처리 기법 중 하나, 어두운 곳에서 잘 인식하게 했음.)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    frame_gray = clahe.apply(frame_gray)

    faces = face_cascade.detectMultiScale(frame_gray)

    if len(faces) == 0:
        # 얼굴이 감지되지 않았을 때
        cv2.putText(show_frame, "No face detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        cv2.imshow(title_name, show_frame)
        return
    
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
        points = np.matrix([[p.x, p.y] for p in predictor(image, rect).parts()])
        show_parts = points[EYES]
        right_eye_EAR = getEAR(points[RIGHT_EYE])
        left_eye_EAR = getEAR(points[LEFT_EYE])
        mean_eye_EAR = (right_eye_EAR + left_eye_EAR) / 2 

        right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype("int")
        left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype("int")

        cv2.putText(image, "{:.2f}".format(right_eye_EAR), (right_eye_center[0, 0], right_eye_center[0, 1] + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(image, "{:.2f}".format(left_eye_EAR), (left_eye_center[0, 0], left_eye_center[0, 1] + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        for (i, point) in enumerate(show_parts):
            x = point[0, 0]
            y = point[0, 1]
            cv2.circle(image, (x, y), 1, (0, 255, 255), -1)
            
        if mean_eye_EAR > min_EAR:
            color = (0, 255, 0)
            status = 'Awake'
            number_closed = number_closed - 1
            if number_closed < 0:
                number_closed = 0
        else:
            color = (0, 0, 255)
            status = 'sleep'
            number_closed += 1

        if number_closed >= closed_limit:
            drowsiness_count += 1  # 졸음 횟수 증가
            number_closed = 0  # 카운트 초기화
                     
        sign = f'sleep count: {number_closed} / {closed_limit}, drowsiness count: {drowsiness_count}'
        break  # 얼굴 하나만 처리

    cv2.putText(show_frame, status, (10, 60), cv2.FONT_HERSHEY_DUPLEX, 1, color, 2)
    cv2.putText(show_frame, sign, (10, frame_height - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    cv2.imshow(title_name, show_frame)

cap = cv2.VideoCapture(0)
time.sleep(2.0)
if not cap.isOpened():
    print('Could not open video')
    exit(0)

while True:
    ret, frame = cap.read()

    if frame is None:
        print('Could not read frame')
        cap.release()
        break

    detectAndDisplay(frame)
    
    # q 입력시 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@170.523] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
